In [7]:
import IndicTransToolkit
print("IndicTransToolkit imported successfully!")


IndicTransToolkit imported successfully!


In [ ]:
# Print device info only when available
try:
    import torch
    if torch.cuda.is_available():
        print(f"Device: {model.device}")
        print(f"CUDA available: {torch.cuda.is_available()}")
        try:
            print(f"GPU name: {torch.cuda.get_device_name(0)}")
        except Exception:
            # Some environments may not allow querying the device name
            print("GPU present but could not query device name.")
    else:
        print(f"Device: {model.device}")
        print("CUDA available: False")
except NameError:
    # model may not be defined yet if this cell is run out of order
    print("Model is not defined in this scope. Run the model-loading cell first.")


Device: cuda:0
CUDA available: True
GPU name: NVIDIA GeForce RTX 4060 Laptop GPU


In [11]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


In [6]:
from IndicTransToolkit.processor import IndicProcessor

ip = IndicProcessor(inference=True)

sample = ["The lungs appear clear and well expanded."]
batch = ip.preprocess_batch(sample, src_lang="eng_Latn", tgt_lang="hin_Deva")

print("Preprocessed batch:", batch)


Preprocessed batch: ['eng_Latn hin_Deva The lungs appear clear and well expanded .']


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_ID = "facebook/nllb-200-distilled-600M"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID).to(DEVICE)

LANG_CODES = {
    "Hindi": "hin_Deva",
    "Telugu": "tel_Telu",
    "Tamil": "tam_Taml",
    "Marathi": "mar_Deva",
    "Malayalam": "mal_Mlym",
}

def translate(text, target_language):
    tgt = LANG_CODES[target_language]
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        generated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt),
            num_beams=5,
            max_length=256,
            repetition_penalty=1.0,
            no_repeat_ngram_size=3,
        )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

text = "The lungs appear clear and well expanded. No nodules or infiltrates are seen."

for lang in LANG_CODES.keys():
    print(lang, "→", translate(text, lang))


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Hindi → फेफड़े स्पष्ट और अच्छी तरह से विस्तारित दिखाई देते हैं। कोई नोडल या घुसपैठ नहीं दिखाई देती है।
Telugu → ఊపిరితిత్తులు స్పష్టంగా కనిపిస్తాయి మరియు బాగా విస్తరించాయి. నోడుల్స్ లేదా ఇన్ఫిలిట్రేట్లు కనిపించవు.
Tamil → நுரையீரல் தெளிவாகவும் விரிவாகவும் தோன்றுகிறது. எண்கள் அல்லது ஊடுருவல்கள் எதுவும் காணப்படவில்லை.
Marathi → फुफ्फुसे स्पष्ट आणि विस्तारित दिसतात. कोणतेही नोडल्स किंवा घुसखोरी दिसत नाही.
Malayalam → ശ്വാസകോശങ്ങള് വ്യക്തവും നന്നായി വിപുലീകരിച്ചതുമായി കാണപ്പെടുന്നു.
